In [1]:
# 모델 학습시키기
# 모델 학습에 사용할 연합뉴스 데이터셋 다운로드
from datasets import load_dataset
klue_tc_train = load_dataset('klue', 'ynat', split='train')
klue_tc_eval = load_dataset('klue', 'ynat', split='validation')
klue_tc_train

Dataset({
    features: ['guid', 'title', 'label', 'url', 'date'],
    num_rows: 45678
})

In [ ]:
# 데이터 확인
klue_tc_train[0]

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

In [ ]:
# 데이터 확인
klue_tc_train.features['label'].names
# ['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

In [ ]:
# 데이터 전처리 - 불필요한 컬럼 제거
klue_tc_train = klue_tc_train.remove_columns(['guid', 'url', 'date'])
klue_tc_eval = klue_tc_eval.remove_columns(['guid', 'url', 'date'])
klue_tc_train

Dataset({
    features: ['title', 'label'],
    num_rows: 45678
})

In [ ]:
# 데이터 전처리 - 카테고리를 문자로 표기한 label_str 컬럼 추가
klue_tc_train.features['label']
# ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)

klue_tc_train.features['label'].int2str(1)
# '경제'

klue_tc_label = klue_tc_train.features['label']

def make_str_label(batch):
  batch['label_str'] = klue_tc_label.int2str(batch['label'])
  return batch

klue_tc_train = klue_tc_train.map(make_str_label, batched=True, batch_size=1000)

klue_tc_train[0]
# {'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3, 'label_str': '생활문화'}

{'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3, 'label_str': '생활문화'}

In [ ]:
# 학습/검증/테스트 데이터셋 분할 - 데이터 셋 준비
train_dataset = klue_tc_train.train_test_split(test_size=10000, shuffle=True, seed=42)['test'] # 10,000개 추출
dataset = klue_tc_eval.train_test_split(test_size=1000, shuffle=True, seed=42) # 1,000개 추출
test_dataset = dataset['test']
valid_dataset = dataset['train'].train_test_split(test_size=1000, shuffle=True, seed=42)['test'] # 1,000개 추출

In [ ]:
# Trainer를 사용하지 않는 학습: (1) 학습을 위한 모델과 토크나이저 준비 - pre-trained model(klue/roberta-base) 및 AutoTokenizer(텍스트 토큰화)
import torch
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np

def tokenize_function(examples): # 제목(title) 컬럼에 대한 토큰화
    return tokenizer(examples["title"], padding="max_length", truncation=True)

# 모델과 토크나이저 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # GPU 설정
model_id = "klue/roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_dataset.features['label'].names))
tokenizer = AutoTokenizer.from_pretrained(model_id)
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
# Trainer를 사용하지 않는 학습: (2) 학습을 위한 데이터 준비 - 데이터로더 생성
def make_dataloader(dataset, batch_size, shuffle=True):
    dataset = dataset.map(tokenize_function, batched=True).with_format("torch") # 데이터셋에 토큰화 수행
    dataset = dataset.rename_column("label", "labels") # 컬럼 이름 변경
    dataset = dataset.remove_columns(column_names=['title']) # 불필요한 컬럼 제거
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

# 데이터로더 만들기
train_dataloader = make_dataloader(train_dataset, batch_size=8, shuffle=True)
valid_dataloader = make_dataloader(valid_dataset, batch_size=8, shuffle=False)
test_dataloader = make_dataloader(test_dataset, batch_size=8, shuffle=False)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Trainer를 사용하지 않는 학습: (3) 학습을 위한 함수 정의
def train_epoch(model, data_loader, optimizer):
    model.train() # 학습모드 변경
    total_loss = 0
    for batch in tqdm(data_loader):
        optimizer.zero_grad() # 미분 초기화(가중치, 바이어스 파라미터)
        input_ids = batch['input_ids'].to(device) # 모델에 입력할 토큰 아이디
        attention_mask = batch['attention_mask'].to(device) # 모델에 입력할 어텐션 마스크
        labels = batch['labels'].to(device) # 모델에 입력할 레이블
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels) # 모델 계산
        loss = outputs.loss # 손실값 계산
        loss.backward() # 오차역전파, 미분 연산
        optimizer.step() # 오차역전파, 미분 연산 후 모델 가중치, 바이어스 파라미터 업데이트
        total_loss += loss.item() # 손실값 누적
    avg_loss = total_loss / len(data_loader) # 평균 손실값 계산
    return avg_loss

In [ ]:
# Trainer를 사용하지 않는 학습: (4) 평가를 위한 함수 정의
def evaluate(model, data_loader):
    model.eval() # 추론모드 변경
    total_loss = 0
    predictions = []
    true_labels = []
    with torch.no_grad(): # 미분 연산 하지 않음(오차역전파 하지 않음)
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels) # 모델 추론
            logits = outputs.logits # 모델 추론 값
            loss = outputs.loss # 손실함수값
            total_loss += loss.item() # 손실함수값 누적
            # 모델 계산 결과의 logits 속성을 가져와 torch.argmax() 함수를 사용해 가장 큰 값으로 예측한 카테고리 정보를 찾음
            preds = torch.argmax(logits, dim=-1)
            predictions.extend(preds.cpu().numpy()) # 추론(예측) 값 -> 넘파이 타입으로 변경
            true_labels.extend(labels.cpu().numpy()) # 정답 값 -> 넘파이 타입으로 변경
    avg_loss = total_loss / len(data_loader) # 평균 손실함수 값 계산
    accuracy = np.mean(np.asarray(predictions) == np.asarray(true_labels)) # 예측값 == 정답값 비교해 정확도를 계산
    return avg_loss, accuracy

In [ ]:
# Trainer를 사용하지 않는 학습: (5) 학습 수행
num_epochs = 1
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 루프
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train_loss = train_epoch(model, train_dataloader, optimizer)
    print(f"Training loss: {train_loss}")
    valid_loss, valid_accuracy = evaluate(model, valid_dataloader)
    print(f"Validation loss: {valid_loss}")
    print(f"Validation accuracy: {valid_accuracy}")

# Testing
_, test_accuracy = evaluate(model, test_dataloader)
print(f"Test accuracy: {test_accuracy}") # 정확도 0.82

Epoch 1/2


  0%|          | 0/1250 [00:00<?, ?it/s]

Training loss: 0.43953326002806425


  0%|          | 0/125 [00:00<?, ?it/s]

Validation loss: 0.7441377874016761
Validation accuracy: 0.764
Epoch 2/2


...

Training loss: 0.32377947511523963


  0%|          | 0/125 [00:00<?, ?it/s]

Validation loss: 0.6956408118903636
Validation accuracy: 0.792


  0%|          | 0/125 [00:00<?, ?it/s]

Test accuracy: 0.815


In [74]:
import torch
from torch.nn.functional import softmax
from transformers import AutoModelForSequenceClassification, AutoTokenizer

class CustomPipeline:
    def __init__(self, model_id):
        self.model = AutoModelForSequenceClassification.from_pretrained(model_id)
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model.eval()

    def __call__(self, texts):
        # tokenizer 통한 토큰화
        tokenized = self.tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

        with torch.no_grad(): # 미분 연산하지 않음
            outputs = self.model(**tokenized) # 모델 추론
            logits = outputs.logits # 모델 추론 값

        # 가장 큰 예측 확률을 갖는 클래스를 추출해서 반환
        probabilities = softmax(logits, dim=-1)
        scores, labels = torch.max(probabilities, dim=-1)
        labels_str = [self.model.config.id2label[label_idx] for label_idx in labels.tolist()]

        return [{"label": label, "score": score.item()} for label, score in zip(labels_str, scores)]

custom_pipeline = CustomPipeline(model_id)
custom_pipeline(dataset['test']['title'][:100])

[{'label': '사회', 'score': 0.4837909936904907},
 {'label': '사회', 'score': 0.4970826804637909},
 {'label': '스포츠', 'score': 0.5287860631942749},
 {'label': '사회', 'score': 0.930154025554657},
 {'label': '사회', 'score': 0.8547390103340149},
 {'label': 'IT과학', 'score': 0.8367857336997986},
 {'label': '세계', 'score': 0.6763783097267151},
 {'label': 'IT과학', 'score': 0.5875315070152283},
 {'label': '생활문화', 'score': 0.8966208696365356},
 {'label': 'IT과학', 'score': 0.27729278802871704},
 {'label': 'IT과학', 'score': 0.9353365302085876},
 {'label': 'IT과학', 'score': 0.7597654461860657},
 {'label': '세계', 'score': 0.76815265417099},
 {'label': 'IT과학', 'score': 0.9492108225822449},
 {'label': '정치', 'score': 0.9910655617713928},
 {'label': '세계', 'score': 0.9625180959701538},
 {'label': '사회', 'score': 0.9402467608451843},
 {'label': '사회', 'score': 0.8627825975418091},
 {'label': '경제', 'score': 0.9638135433197021},
 {'label': '사회', 'score': 0.8906746506690979},
 {'label': '생활문화', 'score': 0.9825164079666138}